In [1]:
n_qubits = 3

## Step 1: Create Grover n-qubit, compile to L3, and save ISA (QPY)

- 连接 IBM Quantum 并选择 `quantum_config.json` 指定的后端；
- 构建 `grover_algorithm.py` 的 n 比特 Grover 电路；
- 使用 preset pass manager 在目标后端上编译到 L3；
- 将编译后的 ISA 电路保存为 QPY 到 `artifacts/`，并输出保存路径。


In [15]:
from __future__ import annotations

from datetime import datetime

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit import qpy

from ibm_quantum_connector import QuantumServiceManager
from grover_algorithm import grover_algorithm

# Connect and select backend
svc = QuantumServiceManager(config_file='quantum_config.json')
assert svc.connect(), 'Failed to connect to IBM Quantum service'
backend = svc.select_backend()
assert backend is not None, 'Failed to select backend'

# Build source circuit and compile to L3
src_qc = grover_algorithm(n_qubits)
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)
isa = pm.run(src_qc)

# Save ISA QPY
from pathlib import Path
artifacts = Path('artifacts')
artifacts.mkdir(exist_ok=True)

ts = datetime.now().strftime('%Y%m%d_%H%M%S')
qpy_path = artifacts / f'grover{n_qubits}_l3_{backend.name}_{ts}.qpy'
with open(qpy_path, 'wb') as f:
    qpy.dump([isa], f)

print('Saved ISA QPY to:', qpy_path)
isa


num_iterations: 1
Saved ISA QPY to: artifacts/grover3_l3_ibm_brisbane_20251022_150954.qpy


In [16]:
# Draw the L3 ISA circuit and save a PNG next to the QPY
import matplotlib.pyplot as plt
from pathlib import Path

try:
    fig = isa.draw(output='mpl')
    png_path = Path(str(qpy_path).replace('.qpy', '.png'))
    fig.savefig(png_path, dpi=180, bbox_inches='tight')
    print('Saved circuit PNG to:', png_path)
    plt.show()
except NameError as e:
    raise RuntimeError('Please run Step 1 cell first to create `isa` and `qpy_path`.') from e


Saved circuit PNG to: artifacts/grover3_l3_ibm_brisbane_20251022_150954.png


In [2]:
from qiskit import qpy
import glob

# Find the latest qpy file in the artifacts directory
qpy_files = sorted(glob.glob('artifacts/*.qpy'), reverse=True)
if not qpy_files:
    raise FileNotFoundError("No .qpy files found in the artifacts directory.")
qpy_path = qpy_files[0]

with open(qpy_path, 'rb') as f:
    isa = list(qpy.load(f))[0]


In [3]:
from quantum_executor import QuantumExecutor
qe = QuantumExecutor( config_file='quantum_config.json')

qe.run_circuit(isa_circuit=isa, shots=1024, execution_type='ideal_simulator')

{'success': True,
 'execution_type': 'ideal_simulator',
 'backend': 'ibm_brisbane',
 'job_id': None,
 'counts': {'000': 34,
  '001': 31,
  '010': 19,
  '011': 39,
  '100': 23,
  '101': 34,
  '110': 27,
  '111': 817},
 'probabilities': {'000': 0.033203125,
  '001': 0.0302734375,
  '010': 0.0185546875,
  '011': 0.0380859375,
  '100': 0.0224609375,
  '101': 0.033203125,
  '110': 0.0263671875,
  '111': 0.7978515625},
 'shots': 1024,
 'method': 'Aer SamplerV2'}

In [4]:
qe.run_circuit(isa_circuit=isa, shots=1024, execution_type='noisy_simulator')

{'success': True,
 'execution_type': 'noisy_simulator',
 'backend': 'ibm_brisbane',
 'job_id': None,
 'counts': {'000': 45,
  '001': 56,
  '010': 48,
  '011': 67,
  '100': 53,
  '101': 67,
  '110': 68,
  '111': 620},
 'probabilities': {'000': 0.0439453125,
  '001': 0.0546875,
  '010': 0.046875,
  '011': 0.0654296875,
  '100': 0.0517578125,
  '101': 0.0654296875,
  '110': 0.06640625,
  '111': 0.60546875},
 'shots': 1024,
 'method': 'Aer SamplerV2 (from_backend + explicit NoiseModel)'}

In [5]:
qe.run_circuit(isa_circuit=isa, shots=1024, execution_type='real_device')

{'success': True,
 'execution_type': 'real_device',
 'backend': 'ibm_brisbane',
 'job_id': 'd3slfg9sg33c73ddhdog',
 'counts': {'000': 55,
  '001': 70,
  '010': 92,
  '011': 106,
  '100': 52,
  '101': 80,
  '110': 137,
  '111': 432},
 'probabilities': {'000': 0.0537109375,
  '001': 0.068359375,
  '010': 0.08984375,
  '011': 0.103515625,
  '100': 0.05078125,
  '101': 0.078125,
  '110': 0.1337890625,
  '111': 0.421875},
 'shots': 1024,
 'method': 'Runtime SamplerV2 (job mode, ISA only)'}

## Step 2

In [ ]:
from quantum_executor import QuantumExecutor
from delta_debug import run_delta_debug_on_isa

# Step 2: Run Delta Debugging on the compiled ISA circuit `isa`
logical_n_qubits = n_qubits
marked_states = [2**logical_n_qubits - 1]

qe = QuantumExecutor(config_file='quantum_config.json')
result = run_delta_debug_on_isa(
    executor=qe,
    isa_circuit=isa,
    n_qubits=logical_n_qubits,
    marked_states=marked_states,
    tolerance=0.05,
)

# Report spike (sudden error increase) layer if detected
scan = result.get('loss_scan', {})
spike_idx = scan.get('spike_index')
spike_delta = scan.get('spike_delta')
if spike_idx is not None:
    print(f"Detected spike at layer index {spike_idx} (Δloss={spike_delta:.4f})")
else:
    print("No obvious spike detected in loss curve.")

result

No obvious spike detected in loss curve.


{'meta': {'timestamp': '2025-10-22T15:59:01.404580',
  'logical_n_qubits': 3,
  'bit_endianness': 'little (Qiskit bitstrings), reversed to logical order',
  'evaluation': {'shots_ddmin': 2048,
   'tolerance_base': 0.02,
   'tolerance_mode': 'adaptive_2sigma',
   'noisy_retry': 1,
   'shots_prefix_scan': 512}},
 'total_segments': 46,
 'problematic_segments': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45],
 'analysis': {'segment_count': 46,
  'segments': [{'layer_id': 0,
    'description': '3×rz, 2×sx',
    'instructions': 5,
    'operations': [{'operation': 'rz',
      'qubits': [57],
      'params': [1.5707963267948966]},
     {'operation': 'sx', 'qubits': [57], 'params': []},
     {'operation': 'rz', 'qubits': [57], 'params': [1.5707963267948966]},
     {'operation': 'rz